In [ ]:
# Author : Raude Killian
# Last modified 16.07.205


In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import warnings
%matplotlib inline


RECALL_EVENT_DICTIONARY = {
    'onset_fixation_cross': 10,

    'congruent_picture_pres': 31,
    'incongruent_picture_pres': 32,
    'new_picture_pres': 33,

    'onset_question': 4,

    'old-left_keypress': 41,
    'old-right_keypress': 42,
    'old-?_keypress': 43,
    'new_keypress': 44,

    #'onset_final_rating1': 5,
    #'onset_final_rating2': 6,
    'onset_resting_state': 8,
}

RESULTS_TO_ID_DICTIONARY = {
    "correctold-correctlocation_congruent": 61,
    "correctold-incorrectlocation_congruent": 62,
    "correctold-unknowlocation_congruent": 63,

    "correctold-correctlocation_incongruent": 64,
    "correctold-incorrectlocation_incongruent": 65,
    "correctold-unknowlocation_incongruent": 66,

    "correctnew-cr": 71,
    "incorrectnew-fa": 72,
    "incorrectold-miss_congruent": 73,
    "incorrectold-miss_incongruent": 74,
    "too-long": 75,
}

PARTICIPANTS_FILE = r"L:\Common\Users\Qiaoyue\MEG_project\Data\participants.csv"

# MEG recordings might skip or add a few unintentional events, this function check and correct for these using the behavioral output from the experiment.
def check_event_sequence(events):
    # Pattern matchers
    def is_10(e): return e == 10
    def is_3X(e): return 30 <= e < 40
    def is_4(e):  return e == 4
    def is_4X(e): return 40 <= e < 50

    pattern_checkers = [is_10, is_3X, is_4, is_4X]
    pattern_names = ['10', '3X', '4', '4X']

    step = 0  # Start at the first expected type
    broken_events = []  # List to store events that break the pattern

    for i, event in enumerate(events):
        event_id = event[2]
        if pattern_checkers[step](event_id):
            step = (step + 1) % 4
        elif any(check(event_id) for check in pattern_checkers):
            # Wrong type at this step — pattern break
            print(f"❌ Pattern break at index {i}: expected {pattern_names[step]}, got {event_id}")
            broken_events.append(event)
        else:
            # Not part of the pattern — ignore
            continue

    if broken_events:
        return broken_events
    else:
        print("✅ Pattern completed correctly through all relevant events.")
        return None

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

participants = pd.read_csv(PARTICIPANTS_FILE)  
participants = participants['subID'].astype(str).tolist()


print(f"✅ {len(participants)} participants list : {participants}")

In [ ]:
# For tests purposes
#participants = ["F103"]
#sessions = ["encoding"]

In [ ]:
# Plotting events (only for visualization)
for subID in participants:

    plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
    os.makedirs(plots_folder, exist_ok=True)

    intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
    #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
    os.makedirs(intermediates_folder, exist_ok=True)

    final_folder = rf"C:\Users\killg\Desktop\finals\{subID}"
    #final_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\finals\{subID}"
    os.makedirs(final_folder, exist_ok=True)

    # Declare filenames 
    processed_meg_filename = rf"{intermediates_folder}/{subID}_{session}_icaed.fif"
    bhv_results_filename = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\bhv\{subID}\Step2\{subID}_merged_data.csv"

    if not (os.path.exists(processed_meg_filename) and os.path.exists(bhv_results_filename)):
        print(f"❌ Missing file(s) for participant {subID}. Skipping...")
        continue
    print(f"Preprocessing: {subID}_recall_session")

    # Open behavioral performance file
    df = pd.read_csv(bhv_results_filename)  
    bhv_results = df['trialtypeMEG'].astype(str).tolist()
    bhv_results_ID = [RESULTS_TO_ID_DICTIONARY[i] for i in bhv_results]

    # Open processed meg file and extract events
    processed_meg_file = mne.io.read_raw_fif(processed_meg_filename, preload=True, verbose=False)
    events = mne.find_events(
        processed_meg_file, 
        stim_channel=['STI102'],
        shortest_event=1 / processed_meg_file.info['sfreq'],
        verbose=False
    )

    # Extract the picture presentation events with miss_stim correction
    picture_presentation_events = events[np.isin(events[:, 2], [31, 32, 33])]
    if len(picture_presentation_events) == 180:
        print(f"{subID}_recall_session : Correct number of trials")
    else:
        print(f"❌ {subID}_recall_session : Incorrect number of trials \nCorrecting...")
        cleaned_events = events[~np.all(events == check_event_sequence(events), axis=1)]
        picture_presentation_events = cleaned_events[np.isin(cleaned_events[:, 2], [31, 32, 33])]

    picture_presentation_events[:,2] = bhv_results_ID

    # Plot picture presentation events
    sfreq = processed_meg_file.info['sfreq'] 
    fig = mne.viz.plot_events(picture_presentation_events, event_id=RESULTS_TO_ID_DICTIONARY, show=False, on_missing="ignore")
    ax = fig.axes[0]
    xticks = ax.get_xticks()
    xtick_labels_minutes = [f"{(x / sfreq) / 60:.0f}" for x in xticks]

    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels_minutes)
    ax.set_xlabel("Time (minutes)")

    fig.suptitle("Picture Presentation Events", fontsize=14)

    fig.set_size_inches(12, 6)
    fig.savefig(f"{plots_folder}/{subID}_events_picpres_plots.png")
    plt.close(fig) 

    # Plot full events
    fig = mne.viz.plot_events(events, event_id=RECALL_EVENT_DICTIONARY, show=False, on_missing="ignore")
    ax = fig.axes[0]
    xticks = ax.get_xticks()
    xtick_labels_minutes = [f"{(x / sfreq) / 60:.0f}" for x in xticks]

    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels_minutes)
    ax.set_xlabel("Time (minutes)")

    fig.suptitle("All Events", fontsize=14)

    fig.set_size_inches(12, 6)
    fig.savefig(f"{plots_folder}/{subID}_events_full_plots.png")
    plt.close(fig)

    print(f"✅ Participant: {subID} sucessfully completed !  \n")

In [ ]:
# Extract events and save epochs/evoked
numbers_correctcongruent_trials = []
numbers_correctincongruent_trials = []
numbers_correctnew_trials = []
numbers_correctold_trials = []

for subID in participants:

    plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
    os.makedirs(plots_folder, exist_ok=True)

    intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
    #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
    os.makedirs(intermediates_folder, exist_ok=True)

    final_folder = rf"C:\Users\killg\Desktop\finals\{subID}"
    #final_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\finals\{subID}"
    os.makedirs(final_folder, exist_ok=True)

    # Declare filenames 
    processed_meg_filename = rf"{intermediates_folder}/{subID}_{session}_icaed.fif"
    bhv_results_filename = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\bhv\{subID}\Step2\{subID}_merged_data.csv"

    if not (os.path.exists(processed_meg_filename) and os.path.exists(bhv_results_filename)):
        print(f"❌ Missing file(s) for participant {subID}. Skipping...")
        continue
    print(f"Preprocessing: {subID}_recall_session")

    # Open behavioral performance file
    df = pd.read_csv(bhv_results_filename)  
    bhv_results = df['trialtypeMEG'].astype(str).tolist()
    bhv_results_ID = [RESULTS_TO_ID_DICTIONARY[i] for i in bhv_results]

    # Open processed meg file and extract events
    processed_meg_file = mne.io.read_raw_fif(processed_meg_filename, preload=True, verbose=False)
    events = mne.find_events(
        processed_meg_file, 
        stim_channel=['STI102'],
        shortest_event=1 / processed_meg_file.info['sfreq'],
        verbose=False
    )

    # Extract the picture presentation events with miss_stim correction
    picture_presentation_events = events[np.isin(events[:, 2], [31, 32, 33])]
    if len(picture_presentation_events) == 180:
        print(f"{subID}_recall_session : Correct number of trials")
    else:
        print(f"❌ {subID}_recall_session : Incorrect number of trials \nCorrecting...")
        cleaned_events = events[~np.all(events == check_event_sequence(events), axis=1)]
        picture_presentation_events = cleaned_events[np.isin(cleaned_events[:, 2], [31, 32, 33])]

    picture_presentation_events[:,2] = bhv_results_ID

    epochs = mne.Epochs(
        processed_meg_file, 
        events=picture_presentation_events,
        event_id = RESULTS_TO_ID_DICTIONARY,   
        tmin=-0.5,           
        tmax=2,            
        baseline=None,  
        preload=True,
        verbose=False,
        on_missing='ignore'       
    )
    correctold_conditons = [
    "correctold-correctlocation_congruent",
    "correctold-incorrectlocation_congruent",
    "correctold-unknowlocation_congruent",
    "correctold-correctlocation_incongruent",
    "correctold-incorrectlocation_incongruent",
    "correctold-unknowlocation_incongruent",
    ]
        

    count_correctcongruent_trials = len(epochs['correctold-correctlocation_congruent'])
    count_correctincongruent_trials = len(epochs['correctold-correctlocation_incongruent'])
    count_correctnew_trials = len(epochs['correctnew-cr'])
    count_correctold_trials = len(epochs[correctold_conditons])

    numbers_correctcongruent_trials.append(count_correctcongruent_trials)
    numbers_correctincongruent_trials.append(count_correctincongruent_trials)
    numbers_correctnew_trials.append(count_correctnew_trials)
    numbers_correctold_trials.append(count_correctold_trials)

    print(f"Number of correct-congruent trials: {count_correctcongruent_trials}")
    print(f"Number of correct-incongruent trials: {count_correctincongruent_trials}")
    print(f"Number of correct-new trials: {count_correctnew_trials}")
    print(f"Number of correct-old trials: {count_correctold_trials}")

    epochs['correctold-correctlocation_congruent'].save(rf"{final_folder}/{subID}_epochs_correctcong.fif", overwrite=True, verbose=False)
    epochs['correctold-correctlocation_incongruent'].save(rf"{final_folder}/{subID}_epochs_correctincong.fif", overwrite=True, verbose=False)
    epochs['correctnew-cr'].save(rf"{final_folder}/{subID}_epochs_correctnew.fif", overwrite=True, verbose=False)
    epochs[correctold_conditons].save(rf"{final_folder}/{subID}_epochs_correctold.fif", overwrite=True, verbose=False)

    # Average trials, interpolate bads, apply baseline correction in this order
    evoked_cong = epochs['correctold-correctlocation_congruent'].copy().average(method='mean')
    evoked_cong.interpolate_bads(reset_bads=True, verbose = False)
    evoked_cong.apply_baseline((-0.2,0),verbose = False)
    evoked_cong.save(rf"{final_folder}/{subID}_cong_evoked.fif", overwrite=True, verbose=False)

    evoked_incong = epochs['correctold-correctlocation_incongruent'].copy().average(method='mean')
    evoked_incong.interpolate_bads(reset_bads=True, verbose = False)
    evoked_incong.apply_baseline((-0.2,0),verbose = False)
    evoked_incong.save(rf"{final_folder}/{subID}_incong_evoked.fif", overwrite=True, verbose=False)

    evoked_new = epochs['correctnew-cr'].copy().average(method='mean')
    evoked_new.interpolate_bads(reset_bads=True, verbose = False)
    evoked_new.apply_baseline((-0.2,0),verbose = False)
    evoked_new.save(rf"{final_folder}/{subID}_new_evoked.fif", overwrite=True, verbose=False)

    evoked_old = epochs[correctold_conditons].copy().average(method='mean')
    evoked_old.interpolate_bads(reset_bads=True, verbose = False)
    evoked_old.apply_baseline((-0.2,0),verbose = False)
    evoked_old.save(rf"{final_folder}/{subID}_old_evoked.fif", overwrite=True, verbose=False)

    print(f"✅ Participant: {subID} sucessfully completed !  \n")

#To check in R instead for complementary results slides 
print(f"Average number of correct-congruent trials: {np.mean(numbers_correctcongruent_trials):.2f} ± {np.std(numbers_correctcongruent_trials):.2f}")
print(f"Average number of correct-incongruent trials: {np.mean(numbers_correctincongruent_trials):.2f} ± {np.std(numbers_correctincongruent_trials):.2f}")
print(f"Average number of correct-new trials: {np.mean(numbers_correctnew_trials):.2f} ± {np.std(numbers_correctnew_trials):.2f}")
print(f"Average number of correct-old trials: {np.mean(numbers_correctold_trials):.2f} ± {np.std(numbers_correctold_trials):.2f}")